In [13]:
!pip install --upgrade selenium

In [19]:
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [21]:
!ls /usr/lib/chromium-browser/chromedriver

chromedriver


In [151]:
from google.colab import userdata
email= userdata.get('email')
password= userdata.get('password')

In [152]:
url= 'https://archiveofourown.org/users/Sw33tBl0550m/readings'

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [153]:
driver = web_driver()
driver.get(url)


In [154]:
driver.find_element(By.XPATH, '//*[@id="user_login"]').send_keys(email)
driver.find_element(By.XPATH, '//*[@id="user_password"]').send_keys(password)
driver.find_element(By.XPATH, '//*[@id="new_user"]/dl/dd[4]/input').click()

In [155]:
import pandas as pd
import time
def scrape_titles_two_pages(driver):
    titles = []
    links= []
    authors= []
    fandom_tags= []
    relationship_tags= []
    character_tags= []
    freeform_tags= []
    summary= []
    words= []
    chapters= []
    latest_view= []
    current_page = 1
    max_pages = 4

    while current_page <= max_pages:
        print(f"Scraping page {current_page}")

        # Scrape current page
        # getting titles and links
        header_titles = driver.find_elements(By.CSS_SELECTOR, 'h4.heading a[href^="/works/"]')
        print(f"Found {len(header_titles)} titles on this page")

        for title in header_titles:
            titles.append(title.text)
            links.append(title.get_attribute('href'))


        # getting authors, fandoms tags, relationship tags, character tags
        articles = driver.find_elements(By.CSS_SELECTOR, 'li[role="article"]')
        for article in articles:
            #authors
            author_elements= article.find_elements(By.CSS_SELECTOR, 'h4.heading a[rel="author"]')
            author_list= [author.text for author in author_elements]
            if not author_list:
              authors.append("Anonymous")
            else:
              authors.append(", ".join(author_list))

            # fandoms
            fandom_elements = article.find_elements(By.CSS_SELECTOR, 'h5.fandoms a')
            fandom_list= [fandom.text for fandom in fandom_elements]
            fandom_tags.append(", ".join(fandom_list))

            # relationship tags
            relationship_elements= article.find_elements(By.CSS_SELECTOR, 'li.relationships a')
            relationship_list= [relationship.text for relationship in relationship_elements]
            relationship_tags.append(", ".join(relationship_list))

            # character tags
            character_elements= article.find_elements(By.CSS_SELECTOR, 'li.characters a')
            character_list= [character.text for character in character_elements]
            character_tags.append(", ".join(character_list))

            # freeforms
            freeform_elements= article.find_elements(By.CSS_SELECTOR, 'li.freeforms a')
            freeform_list= [freeform.text for freeform in freeform_elements]
            freeform_tags.append(", ".join(freeform_list))

            # summary
            summary_elements= article.find_elements(By.CSS_SELECTOR, 'blockquote.userstuff p')
            summary_list= [summary.text for summary in summary_elements]
            summary.append("\n\n".join(summary_list))

            # word count
            word_element= article.find_element(By.CSS_SELECTOR,'dd.words')
            words.append(word_element.text)

            # chapter
            chapters_elements= article.find_element(By.CSS_SELECTOR, 'dd.chapters')
            chapters.append(chapters_elements.text)

            # view history
            latest_view_elements= article.find_element(By.CSS_SELECTOR, 'div.user.module.group h4.viewed.heading')
            latest_view.append(latest_view_elements.text)

        print(f"Found {len(authors)} authors on this page")

        print(f"Found {len(fandom_tags)} fandom tags on this page")

        print(f"Found {len(relationship_tags)} relationship tags on this page")

        print(f"Found {len(character_tags)} character tags on this page")

        print(f"Found {len(freeform_tags)} character tags on this page")

        print(f"Found summaries on this page")

        print(f"Found words on this page")

        print(f"Found chapter on this page")




        if current_page == max_pages:
            break

        try:
            # Add explicit wait for next button
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[rel="next"]'))
            )
            print("Found next button, attempting to click")

            # Add a small delay before clicking
            time.sleep(2)

            next_button.click()
            print("Clicked next button")

            # Wait for next page to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h4.heading'))
            )
            current_page += 1
            print(f"Successfully moved to page {current_page}")

        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    df= pd.DataFrame({"Title": titles, 'Author': authors, "Fandom Tags": fandom_tags, "Relationship Tags":relationship_tags, "Character Tags":character_tags,
                      "Freeform Tags": freeform_tags, "Summary":summary, "Chapters": chapters, "Words": words, "Latest View": latest_view, "Link":links})

    return df

# Use the function
all_titles = scrape_titles_two_pages(driver)
print(f"\nTotal titles collected: {len(all_titles)}")


Scraping page 1
Found 20 titles on this page
Found 20 authors on this page
Found 20 fandom tags on this page
Found 20 relationship tags on this page
Found 20 character tags on this page
Found 20 character tags on this page
Found summaries on this page
Found words on this page
Found chapter on this page
Found next button, attempting to click
Clicked next button
Successfully moved to page 2
Scraping page 2
Found 20 titles on this page
Found 40 authors on this page
Found 40 fandom tags on this page
Found 40 relationship tags on this page
Found 40 character tags on this page
Found 40 character tags on this page
Found summaries on this page
Found words on this page
Found chapter on this page
Found next button, attempting to click
Clicked next button
Successfully moved to page 3
Scraping page 3
Found 20 titles on this page
Found 60 authors on this page
Found 60 fandom tags on this page
Found 60 relationship tags on this page
Found 60 character tags on this page
Found 60 character tags on thi

In [156]:
all_titles.to_csv("My_AO3_data.csv", index=False)